<a href="https://colab.research.google.com/github/kcarmona1/Coffee-Shop-Analysis/blob/main/392_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

%matplotlib inline

# Coffee Shop Data

Karla Carmona | CPSC-392-04 | Final Project Analysis

### Set Up

Data Retrieved From:

https://www.kaggle.com/datasets/ylchang/coffee-shop-sample-data-1113

In [ ]:
df1 = pd.read_csv("https://raw.githubusercontent.com/kcarmona1/datasets/main/201904%20sales%20reciepts.csv")
del df1["line_item_id"]
del df1["sales_outlet_id"]
del df1["staff_id"]
del df1["transaction_time"]
del df1["transaction_id"]
del df1["order"]
df1.head()


,transaction_date,customer_id,instore_yn,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,2019-04-01,558,N,52,1,2.50,2.50,N
1,2019-04-01,781,N,27,2,7.00,3.50,N
2,2019-04-01,788,Y,46,2,5.00,2.50,N
3,2019-04-01,683,N,23,2,5.00,2.50,N
4,2019-04-01,99,Y,34,1,2.45,2.45,N


In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/kcarmona1/datasets/main/customer.csv")
del df2["loyalty_card_number"]
del df2["customer_first-name"]
del df2["customer_email"]
del df2["home_store"]
df2.head()

,customer_id,customer_since,birthdate,gender,birth_year
0,1,2017-01-04,1950-05-29,M,1950
1,2,2017-01-07,1950-07-30,M,1950
2,3,2017-01-10,1950-09-30,M,1950
3,4,2017-01-13,1950-12-01,M,1950
4,5,2017-01-15,1951-02-01,M,1951


In [ ]:
df3 = pd.read_csv("https://raw.githubusercontent.com/kcarmona1/datasets/main/product.csv")
del df3["product_type"]
del df3["product_category"]
del df3["current_wholesale_price"]
df3.head()

,product_id,product_group,product,product_description,unit_of_measure,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,$18.00,Y,N,N
1,2,Whole Bean/Teas,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,$18.00,Y,N,N
2,3,Whole Bean/Teas,Espresso Roast,Our house blend for a good espresso shot.,1 lb,$14.75,Y,N,N
3,4,Whole Bean/Teas,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,$20.45,Y,N,N
4,5,Whole Bean/Teas,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,$15.00,Y,N,N


### Data Cleaning

In [ ]:
q1_df = pd.merge(df1, df2, on="customer_id")
q1_df.head()

,transaction_date,customer_id,instore_yn,product_id,quantity,line_item_amount,unit_price,promo_item_yn,customer_since,birthdate,gender,birth_year
0,2019-04-01,558,N,52,1,2.50,2.50,N,2018-06-19,1983-02-25,F,1983
1,2019-04-01,558,N,29,1,2.50,2.50,N,2018-06-19,1983-02-25,F,1983
2,2019-04-01,558,Y,48,2,5.00,2.50,N,2018-06-19,1983-02-25,F,1983
3,2019-04-02,558,N,60,1,3.75,3.75,N,2018-06-19,1983-02-25,F,1983
4,2019-04-03,558,N,32,1,3.00,3.00,N,2018-06-19,1983-02-25,F,1983


In [ ]:
q1_full_df = pd.merge(q1_df, df3, on="product_id")
q1_full_df.head()

,transaction_date,customer_id,instore_yn,product_id,quantity,line_item_amount,unit_price,promo_item_yn,customer_since,birthdate,gender,birth_year,product_group,product,product_description,unit_of_measure,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,2019-04-01,558,N,52,1,2.5,2.5,N,2018-06-19,1983-02-25,F,1983,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N
1,2019-04-17,38,Y,52,2,5.0,2.5,N,2017-04-16,1956-09-13,M,1956,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N
2,2019-04-13,128,Y,52,2,5.0,2.5,N,2017-12-20,1972-01-05,M,1972,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N
3,2019-04-06,599,Y,52,1,2.5,2.5,N,2018-09-12,1988-05-20,F,1988,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N
4,2019-04-24,152,Y,52,1,2.5,2.5,N,2018-02-25,1976-02-04,M,1976,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N


In [ ]:
q1_full_df["age"] = 2022 - q1_full_df["birth_year"]
q1_full_df.head()

,transaction_date,customer_id,instore_yn,product_id,quantity,line_item_amount,unit_price,promo_item_yn,customer_since,birthdate,...,birth_year,product_group,product,product_description,unit_of_measure,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn,age
0,2019-04-01,558,N,52,1,2.5,2.5,N,2018-06-19,1983-02-25,...,1983,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N,39
1,2019-04-17,38,Y,52,2,5.0,2.5,N,2017-04-16,1956-09-13,...,1956,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N,66
2,2019-04-13,128,Y,52,2,5.0,2.5,N,2017-12-20,1972-01-05,...,1972,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N,50
3,2019-04-06,599,Y,52,1,2.5,2.5,N,2018-09-12,1988-05-20,...,1988,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N,34
4,2019-04-24,152,Y,52,1,2.5,2.5,N,2018-02-25,1976-02-04,...,1976,Beverages,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,$2.50,Y,N,N,46


In [ ]:
# test code to change "current_retail_price" to a continuous, float variable rather than a string
sample = '$2.50 '
float(sample.replace('$',''))

sampleDF = pd.DataFrame({
    "x" : ['$' + str(i) + '.' + str(i) for i in range(5)]
})
sampleDF

for i in range(sampleDF.shape[0]):
  sampleDF.iloc[i, 0] = float(sampleDF.iloc[i, 0].replace('$',''))


sampleDF

,x
0,0.0
1,1.1
2,2.2
3,3.3
4,4.4


In [ ]:
for i in range(q1_full_df.shape[0]):
  q1_full_df.iloc[i, 16] = float(q1_full_df.iloc[i, 16].replace("$",""))

# Analysis

### **Question 1**

When predicting the price a customer paid, what is the difference in R2 scores and MSE when comparing a linear regression model using all predictors against a lasso regression model?

In [1]:
# Data Frame
q1 = q1_full_df.copy()
categoricals = ["instore_yn", "promo_item_yn", "gender", "product_group",
                "tax_exempt_yn", "promo_yn", "new_product_yn"]
# Cleaning and Setup
dummies = pd.get_dummies(q1[categoricals])
q1 = pd.concat([q1, dummies], axis = 1)

contins = ["quantity", "age", "current_retail_price"]

predictors = ['instore_yn_ ', 'instore_yn_N', 'instore_yn_Y', 'promo_item_yn_N',
              'promo_item_yn_Y', 'gender_F', 'gender_M', 'gender_N',
              'product_group_Add-ons', 'product_group_Beverages',
              'product_group_Food', 'product_group_Merchandise',
              'product_group_Whole Bean/Teas', 'tax_exempt_yn_N', 
              'tax_exempt_yn_Y', 'promo_yn_N', 'new_product_yn_N', 
              'quantity', 'age', 'current_retail_price']

# Z-scoring
z = StandardScaler()
q1[contins] = z.fit_transform(q1[contins])

# TTS
X = q1[predictors]
y = q1["unit_price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

lr = LinearRegression()

# Model One (all predictors)
lr.fit(X_train, y_train)

train_r2 = lr.score(X_train, y_train)
test_r2 = lr.score(X_test, y_test)

# Model Two (Lasso)
q1_2 = q1.copy()
q1_2[contins] = z.fit_transform(q1_2[contins])

X2 = q1_2[predictors]
y2 = q1_2["unit_price"]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2)

# Hyperparameter tuning
lsr_tune = LassoCV(cv = 5).fit(X2_train, y2_train)

print("\nWe chose " + str(lsr_tune.alpha_) + " as our alpha.")

# Fitting on Lasso

lsr = Lasso(alpha = 0.002131930146518451)
lsr.fit(X2_train, y2_train)
lasso_train_r2 = lsr.score(X2_train, y2_train)
lasso_test_r2 = lsr.score(X2_test, y2_test)

# MSE's
lsr_mse_test = mean_squared_error(y2_test, lsr.predict(X2_test))
lsr_mse_train = mean_squared_error(y2_train, lsr.predict(X2_train))
lr_mse_test = mean_squared_error(y_test, lr.predict(X_test))
lr_mse_train = mean_squared_error(y_train, lr.predict(X_train))

# Model Success

print("LASSO:")
print("TRAIN R2: ", lasso_train_r2)
print("TEST R2: ", lasso_test_r2)
print("TRAIN MSE: ", lsr_mse_train)
print("TEST MSE: ", lsr_mse_test)
print("--------------------------")
print("LR:")
print("TRAIN R2: ", train_r2)
print("TEST R2: ", test_r2)
print("TRAIN MSE: ", lr_mse_train)
print("TEST MSE: ", lr_mse_test)

NameError: ignored

In [ ]:
r2s = pd.DataFrame({
    "method": ["Lasso Train", "Lasso Test", "LR Train", "LR Test"],
    "R2s": [lasso_train_r2, lasso_test_r2, train_r2, test_r2],
})
(ggplot(r2s, aes(x = "method", y = "R2s", fill = "method")) + 
 geom_col() + theme_minimal() + coord_cartesian(ylim=(0.98, 1)) + 
 labs(title = "R2 Score Comparison") + ylab("Score") + xlab(""))

In [ ]:
mses = pd.DataFrame({
    "method": ["Lasso Train", "Lasso Test", "LR Train", "LR Test"],
    "MSE": [lsr_mse_train, lsr_mse_test, lr_mse_train, lr_mse_test]
})
(ggplot(mses, aes(x = "method", y = "MSE", fill = "method")) + 
 geom_col() + theme_minimal() + 
 labs(title = "Mean Squared Error Comparison") + ylab("MSE") + xlab(""))

**Answer:** 


*   Running a linear regression and LASSO regression model allowed me to pull the R2 and Mean Squared Error scores, which could then be plotted to easily compare the success of both models against each other. As depicted in the graphs, the models are very similar in both R2 and MSE scores. This visualization is insightful because it shows that the linear regression model using all the predictors performs nearly the same as the LASSO regression model which penalizes coefficients and limits them to nearly zero for dimensionality reduction purposes. Overall, the modelling used to answer this question allows me to conclude that LASSO regression is not neccessary to be used for variable selection because doing so did not produce any significant improvement in performance or accuracy of the model, as can be seen through the comparison of MSE and R2 scores between both train and test sets. 



### **Question 2**

What characterizes clusters that emerge when considering retail price and quantity purchased?

In [ ]:
# Data Frame and Set Up
q2 = q1_full_df.copy()

features = ["current_retail_price", "unit_price", "age", "quantity"]
X = q2[features]

# Z Scoring
z = StandardScaler()
X[features] = z.fit_transform(X)

# EM Model
EM = GaussianMixture(n_components = 2)
EM.fit(X)
cluster = EM.predict(X)

print("Silhouette Score: ", silhouette_score(X, cluster))

X["cluster"] = cluster


In [ ]:
# Plot to Visualize Cluster
(ggplot(X, aes(x = "quantity", y = "current_retail_price")) + 
 geom_point() + theme_minimal() + labs(title = "Quantity and Retail Price") + ylab("Current Retail Price") + 
 xlab("Quantity")) 

In [ ]:
# Plot w/ Clusters
(ggplot(X, aes(x = "quantity", y = "current_retail_price", color = "factor(cluster)")) + 
 geom_point() + theme_minimal() + labs(title = "EM Clustering between Quantity and Retail Price") + ylab("Current Retail Price") + 
 xlab("Quantity")) 

**Answer:**


*   The Expectation Maximization with Gaussian Mixture Models resulted in two clusters when ran with the feautres "Quantity" and "Current Retail Price". Ultimately, the clusters that emerged made sense given the context of the data and detail customer purchasing habits as they relate to how much is typically spent and the amount of product being bought. The first cluster presents those transactions in which customers purchased a low quantity of items, and the price of those items ranged quite a bit from less expensive to a bit pricier. The second cluster was less dense and showed customers who purchased more items (higher quantities) that were at a lower cost. This clustering is meaningful in showing that these coffee shops sell more items within a transaction when those items have a lower retail price. Additionally, this modeling reveals that the majority of transactions for this type of business are going to be of low quantities, though they are sold at a range of prices. 



### **Question 3**

How does the average amount spent per transaction differ among varying age groups? What product groups are the most profitable among the highest spending demographics of customers?




In [ ]:
# Set Up
q3 = q1_full_df.copy()
df = q3.groupby(["age"])[["unit_price"]].mean()
df.iloc[49:52]

# Data Frame
q3 = q1_full_df.copy()
means = [ 
    float(df.iloc[0:9].mean()),
    float(df.iloc[9:19].mean()),
    float(df.iloc[19:29].mean()),
    float(df.iloc[29:39].mean()),
    float(df.iloc[39:49].mean()),
    float(df.iloc[49:52].mean())]
ranges = [ 
    "20-29",
    "30-39",
    "40-49",
    "50-59",
    "60-69",
    "70-79"]
avg_spent = pd.DataFrame({
    "age": ranges,
    "avg_unit_price": means
})

In [ ]:
# Plot one
(ggplot(avg_spent, aes(x = "age", y = "avg_unit_price", fill = "age")) + theme_minimal() +
 geom_col() + labs(title = "Average Amount Spent According to Age") + 
 coord_cartesian(ylim=(3.2,3.4)) + ylab("Avg. Unit Price") + xlab("Age"))


In [ ]:
# Plot two
highest_avg = ((q3["age"] > 29) & (q3["age"] < 40)) | ((q3["age"] > 69) & (q3["age"] < 80)) 
df2 = q3.loc[highest_avg]

(ggplot(df2, aes(x = "product_group", fill = "product_group")) + theme_minimal() + 
 geom_bar() + labs(title = "Most Profitable Product Groups Among Highest Spenders") + 
 ylab("Count") + xlab("Product Groups"))


**Answer:** 


*   The average amount spent per age group is pretty standard across age ranges and is around $3.30. Though they are not significantly greater than the other groups, the two highest spending age ranges are customers from the ages of 30-39 and 70-79. Among these customers, the most profitable product group is beverages, followed by food, and then add-ons, whole beans/teas, and finally, merchandise. The visualizations produced clearly demonstrate this information and are important because the business can distinguish which age groups to target for advertising efforts and what product groups would be most beneficial to include. 


Debugging Sources:

https://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data


































In [ ]:
# doesn't show this cells output when downloading PDF
!pip install gwpy &> /dev/null

# installing necessary files
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!sudo apt-get update
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

# installing pypandoc
!pip install pypandoc

# connecting your google drive
from google.colab import drive
drive.mount('/content/drive')

# copying your file over. Change "Class6-Completed.ipynb" to whatever your file is called (see top of notebook)
!cp "drive/My Drive/Colab Notebooks/392_Final_Project.ipynb" ./

# Again, replace "Class6-Completed.ipynb" to whatever your file is called (see top of notebook)
!jupyter nbconvert --to PDF "392_Final_Project.ipynb"